In [1]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics/
    !pip install -r requirements.txt
    !pip install -e .
    %cd notebooks/

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_feather("../data/streamers.feather").set_index("id")
df.head(5)

,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
id,,,,,,,,,,
459331509,auronplay,10874205.0,partner,feliz pero no mucho,es,Minecraft,212166590,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03 14:02:49,"[210708721, 431460701, 77649106, 130065491, 42..."
39276140,Rubius,10483570.0,partner,se contar hasta patata,es,Propnight,222921576,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12 18:46:56,"[91136321, 198363811, 210708721, 133528221, 70..."
48878319,TheGrefg,8708686.0,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Five Nights at Freddy's,197916916,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12 00:45:27,"[143776262, 675347177, 33734881, 248222879, 18..."
83232866,ibai,8536480.0,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286994651,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20 16:47:56,"[35980866, 145908612, 205218019, 91136321, 524..."
121510236,juansguarnizo,6013625.0,partner,"Juan Sebastián Guarnizo Algarra, más conocido ...",es,Just Chatting,127964660,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-04-11 03:34:11,"[43419527, 517536651, 476005292, 109492660, 55..."


In [6]:
G = nx.from_pandas_edgelist(
    df.reset_index().explode("user_follows"),
    source="id",
    target="user_follows",
    create_using=nx.DiGraph()
)
nx.set_node_attributes(G, name='name', values=df.name.to_dict())
nx.set_node_attributes(G, name='num_followers', values=df.num_followers.to_dict())
nx.set_node_attributes(G, name='view_count', values=df.view_count.to_dict())


In [4]:
# checking the nodes of the graph
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges(), "\n")
for i, user in enumerate(list(G.out_degree())):
  if i > 5:
    break
  elif i==5:
    user = "61127118",G.out_degree("61127118")
  print(G.nodes[user[0]]["name"], "follows", user[1], "users")

print()
for i, user in enumerate(list(G.in_degree())+["61127118"]):
  if i > 5:
    break
  elif i==5:
    user = "61127118",G.in_degree("61127118")
  print(G.nodes[user[0]]["name"], "is followed by", user[1], "users in the dataset")

Number of nodes: 75631
Number of edges: 1436042 

auronplay follows 45 users
josecristo_ follows 103 users
xXxTheFocuSxXx follows 90 users
Ampeterby7 follows 114 users
javiDMr10 follows 106 users
DavimenXPro follows 16 users

auronplay is followed by 4044 users in the dataset
josecristo_ is followed by 149 users in the dataset
xXxTheFocuSxXx is followed by 295 users in the dataset
Ampeterby7 is followed by 682 users in the dataset
javiDMr10 is followed by 103 users in the dataset
DavimenXPro is followed by 2 users in the dataset


In [3]:
from src.utils import remove_outside_follows
from src.graph_utils import df_to_nx
small_df = remove_outside_follows(
    df[(~df.user_follows.isnull())&(~df.num_followers.isnull())]
).sort_values("num_followers",ascending=False)
# small_G = df_to_nx(small_df.head(50) )


In [44]:
# Normalize columns of the small_df dataframe
df = small_df.copy()
# df["num_followers"] = df["num_followers"] / df["num_followers"].max()
# df["view_count"] = df["view_count"] / df["view_count"].max()

# plt.figure(figsize=(10,10))
user = "davimenxpro"
is_user = (df.name.str.lower()==user.lower()).head(5000)

# Plot the graph with logarithmic scale for the number of followers
fig = px.scatter(df.head(5000), x="num_followers", y="view_count",
                    hover_name="name", log_x=True, log_y=True,#color=is_user,
                    title=f"View count v Number of followers of the top 5000 Twitch Streamers",
                    labels={"num_followers": "Number of followers", "view_count": "View count"},
                #     color_discrete_sequence=px.colors.sequential.Plasma,
)
# Set figure theme
fig.update_layout(
        template="plotly_dark",
        title_font_size=18,
        title_font_family="Sans serif",
        title_x=0.5, title_xanchor="center",
        font=dict(
                family="Sans serif",
                size=10,
                color="#ffffff"
        ),
        paper_bgcolor="#222222",plot_bgcolor="#222222",
        legend_title_font_size=12, legend_font_size=12,
        legend_font_color="#ffffff",legend_title_font_color="#ffffff",
        legend_orientation="h",
        legend_x=0.5, legend_xanchor="center", 
        legend_y=1.1, legend_yanchor="bottom",
        height=600,
        width=800,
)

#px.scatter(df.head(5000),x="num_followers", y="view_count",color=is_user,hover_data=["name"])

In [6]:
pio.templates.default = "plotly_dark"
bar_of = ["view_count","num_followers"][1]
bar_by = ["last_game_played_name","broadcaster_type","lang"][1]
labels_dict = {
    'num_followers':'Number of Followers',
    'view_count':'View Count',
    'broadcaster_type':'Broadcaster Type',
    'last_game_played_name':'Game Played'
}
if bar_by in ["last_game_played_name","view_count","num_followers"]:
    df = small_df.head(5000).groupby(bar_by).count().reset_index()
    title = f"Count of Streamers by {labels_dict[bar_by]}"
else:
    df = small_df.head(5000).groupby(bar_by).sum().reset_index()
    title=f"{labels_dict.get(bar_of)} by {labels_dict.get(bar_by)}"

df = df[df[bar_of]>5]

for row in df.iloc:
    cat = row[bar_by]
    top_streamer_in_category = small_df[small_df[bar_by]==cat].sort_values(bar_of,ascending=False).iloc[0]['name']
    df.loc[df[bar_by]==cat,'Top Streamer'] = top_streamer_in_category

fig = px.bar(
    df,
    x=bar_by, y=bar_of,
    color=bar_by,
    hover_data=["Top Streamer"],
    title=title,
    labels={bar_of: labels_dict.get(bar_of), bar_by: labels_dict.get(bar_by)},
    height=600,
    log_y=bar_by in ["last_game_played_name","view_count","num_followers"],
)
fig.update_layout(
    paper_bgcolor="#222222",plot_bgcolor="#222222",
)
fig

In [144]:
small_df["created_at"] = small_df.created_at.astype("datetime64")
df = small_df.copy()
df["month"] = small_df.created_at.dt.strftime("%Y-%m")
df["count"] = 1
df = df.groupby("month").sum().reset_index()
df["cumulative_growth"] = df['count'].cumsum()

fig = px.bar(
    df,x="month",y="cumulative_growth",
    title="Cumulative # of Streamers that join the platform over time",
    # hover_data=["Top Streamer"],
    labels={"month": "Month", "cumulative_growth": "Cumulative Growth"},
    height=500,
)

fig.update_layout(
    paper_bgcolor="#222222",plot_bgcolor="#222222",
)

In [148]:
# top n stremers with most follows
top_n = 10
df = small_df.copy().sort_values("num_followers",ascending=False).head(top_n).sort_values("num_followers")

fig = px.bar(
    df,x="name",y="num_followers",
    title=f"Top {top_n} Streamers by Number of Followers",
    labels={"num_followers": "Number of Followers","name": "Name"},
    height=500,
)

fig.update_layout(
    paper_bgcolor="#222222",plot_bgcolor="#222222",
)

In [170]:
df = small_df.head(5000).copy()
fig = px.histogram(df, x="view_count", nbins=50, title="View Count Histogram",log_y=True)
fig.show()

In [5]:
# # default matplotlib style
# plt.style.use('default')

# # set background color of figure to be white
# plt.rcParams['figure.facecolor'] = 'white'

# # groups = [small_G.nodes[n]["name"].lower()=="ibai" for n in small_G.nodes] 
# group_map = {"Ibai":1, "Not ibai":0}

# cmap = mpl.colors.ListedColormap(["tab:blue", "tab:red", ])#"tab:green", "tab:orange"])
# norm = mpl.colors.BoundaryNorm(np.arange(0,cmap.N), cmap.N)

# fig, ax = plt.subplots(1,1,figsize=(8,6))

# # pos = nx.spring_layout(small_G)
# pos = nx.spring_layout(small_G)
# nx.draw_networkx_edges(small_G, pos,node_size=25,alpha=0.2,width=0.5,edge_color='darkgrey', ax=ax)
# nx.draw_networkx_nodes(small_G, pos, node_size=25, cmap=cmap, ax=ax)
# ax.set_title("Graph of the Top 50 streamers");

# cbar = fig.colorbar(ax.collections[0], ticks=np.arange(0.5,cmap.N)/cmap.N,orientation='vertical')
# cbar.set_ticklabels(sorted(group_map,key=lambda k: group_map[k]))
# plt.show()

In [89]:
small_df.loc["210708721"]

name                                                           josecristo_
num_followers                                                     177601.0
broadcaster_type                                                   partner
description              Soy un joven estudiante y maestro de "Un curso...
lang                                                                    es
last_game_played_name                                             VALORANT
view_count                                                         2033552
profile_image_url        https://static-cdn.jtvnw.net/jtv_user_pictures...
created_at                                             2018-04-01 20:31:56
user_follows             [94936224, 36788012, 133614893, 109492660, 402...
Name: 210708721, dtype: object

In [39]:
# from src.utils import remove_outside_follows

# small_df = remove_outside_follows(
#     df[(~df.num_followers.isnull())]
# ).sort_values(["num_followers","view_count"],ascending=False).astype({"id":int})

# # Prepare data to use with Gephi
# nodes_df = small_df.reset_index().drop(columns=["user_follows","profile_image_url","lang"]).rename(columns={"name":"Label","id":"Id"})
# nodes_df.to_csv("../data/nodes.csv",index=False)

# edges_df = small_df[["user_follows"]].explode("user_follows")\
#     .reset_index().rename(columns={"user_follows":"target","id":"source"})\
#         .astype({"source":int,"target":int})\
#             .dropna()
# edges_df.to_csv("../data/edges.csv",index=False)

In [8]:
df

,broadcaster_type,num_followers,view_count,Top Streamer
0,affiliate,46315319.0,411499270,robertpg
1,partner,386017824.0,8842076952,auronplay


In [10]:
fig = px.pie(values=df.num_followers.tolist(), names=df.broadcaster_type.tolist(), title="Games played")


In [11]:
import pickle
pickle.load(open("/home/simon/Documents/Uni/WebAnalytics/TwitchProject/data/fundamental_metrics/indegree.pkl","rb"))

{'83232866': 0.6508182935506824,
 '39276140': 0.5459602820363542,
 '459331509': 0.49214050067921594,
 '48878319': 0.40798240507147937,
 '36473331': 0.3053237596222265,
 '31919607': 0.2773141859111197,
 '512977322': 0.26476486189274856,
 '57793021': 0.23442654764215018,
 '123922797': 0.2291868814282942,
 '70357283': 0.22899281971666988,
 '121510236': 0.21922504689824696,
 '170079505': 0.20965133579144835,
 '536794313': 0.20499385471246523,
 '47939440': 0.195032020182418,
 '61504845': 0.1905686008150592,
 '22346597': 0.18830454751277573,
 '90075649': 0.1844880005174979,
 '541318059': 0.17581991073161266,
 '96116107': 0.17569053625719644,
 '142932807': 0.16851025292709748,
 '101448647': 0.16495245488065205,
 '76385901': 0.16411152079694674,
 '36138196': 0.16359402289928196,
 '114635439': 0.1617180930202471,
 '494026769': 0.15964810142958794,
 '152633332': 0.15796623326217737,
 '167189231': 0.155766867197102,
 '270186408': 0.15427906074131573,
 '430476278': 0.1509800116437027,
 '143737983'